In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

csv = pd.read_csv("bmi.csv")
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
# thin : [1,0,0], normal : [0,1,0], fat : [0,0,1]
bclass = {"thin" : [1,0,0], "normal" : [0,1,0], "fat" : [0,0,1]}
csv["label_onehat"] = csv["label"].apply(lambda x: np.array(bclass[x]))

print(csv)

       height  weight   label label_onehat
0       0.710    0.62     fat    [0, 0, 1]
1       0.710    0.73     fat    [0, 0, 1]
2       0.885    0.61  normal    [0, 1, 0]
3       0.935    0.48    thin    [1, 0, 0]
4       0.765    0.60     fat    [0, 0, 1]
5       0.930    0.59    thin    [1, 0, 0]
6       0.830    0.53  normal    [0, 1, 0]
7       0.965    0.56    thin    [1, 0, 0]
8       0.920    0.43    thin    [1, 0, 0]
9       0.965    0.64    thin    [1, 0, 0]
10      0.775    0.69     fat    [0, 0, 1]
11      0.710    0.63     fat    [0, 0, 1]
12      0.905    0.72  normal    [0, 1, 0]
13      0.945    0.62    thin    [1, 0, 0]
14      0.700    0.62     fat    [0, 0, 1]
15      0.765    0.37    thin    [1, 0, 0]
16      0.655    0.55     fat    [0, 0, 1]
17      0.685    0.68     fat    [0, 0, 1]
18      0.765    0.40    thin    [1, 0, 0]
19      0.680    0.50     fat    [0, 0, 1]
20      0.625    0.73     fat    [0, 0, 1]
21      0.960    0.71  normal    [0, 1, 0]
22      0.7

In [5]:
test_csv = csv[15000:20000]
test_list  = test_csv[["weight","height"]]
test_onehat = list(test_csv["label_onehat"])

x = tf.placeholder(tf.float32, [None, 2])
y_ = tf.placeholder(tf.float32, [None, 3])

W = tf.Variable(tf.zeros([2,3]))
b = tf.Variable(tf.zeros([3]))

y = tf.nn.softmax(tf.matmul(x,W) + b)

cross_entropy = -tf.reduce_sum(y_*tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)


In [14]:
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(3500):
    i = (step *100) % 14000
    rows = csv[1+i:1+i+100]
    x_pat = rows[["weight", "height"]]
    y_ans = list(rows["label_onehat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict = fd)
    if step %500 == 0:
        cre = sess.run(cross_entropy, feed_dict = fd)
        acc = sess.run(accuracy, feed_dict = {x:test_list, y_:test_onehat})
        print("step=", step, "cre=",cre,"acc=",acc)
acc = sess.run(accuracy, feed_dict={x:test_list, y_:test_onehat})
print("정답률=",acc)

tw = tf.train.SummaryWriter("log_dir", graph=sess.graph)

step= 0 cre= 108.663 acc= 0.3242
step= 500 cre= 57.5887 acc= 0.8904
step= 1000 cre= 45.0209 acc= 0.898
step= 1500 cre= 41.6543 acc= 0.9566
step= 2000 cre= 34.664 acc= 0.943
step= 2500 cre= 34.287 acc= 0.9674
step= 3000 cre= 26.8808 acc= 0.9726
정답률= 0.9712
